<a href="https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/Unit_Process_Designs/Entrance_tank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrance Tank: Executive Summary

### July 5, 2020 

**Entrance Tank Size**

The trash rack and plate settlers in the entrance tank at Gracias have been unsuccessful at capturing grit and preventing it from traveling to the rest of the plant. This grit causes maintenance issues by settling in the flocculator and the channel leading to the sedimentation tanks. To solve this problem, the plate settlers could be replaced with a more traditional horizontal settling basin.

The required plan-view area of a horizontal settling basin is determined by the terminal velocity of the smallest particle it is designed to remove. Kawamura suggests that sand and silt smaller than 0.1 mm do not pose a significat risk to downstream pumps and processes (pg. 416). **However, to settle particles larger than 0.1 mm in diameter, the entrance tank would need to be 11.13 m in length.** This calculation was done under the assumption of a 60 L/s flow rate, and that the entrance tank is the same width as the flocculator. For comparison, a single treatment train at the Gracias plant (excluding the filters) covers an approximate 15 m x 7 m area. Increasing the area of the entrance tank would drive up construction costs and may block plant operator access to important parts of the plant. 

The required plan-view area of the entrance tank could be reduced by increasing the critical particle size that it is designed to capture. Any particles that are not captured in the entrance tank must be scoured in the flocculator and the inlet channel of the sedimentation tanks, otherwise they will cause maintenance issues. Calculations should focus on the inlet channel of the sedimentation tanks since it has lower velocities and less shear than the flocculator. 

Two approaches were taken to calculate the smallest particle that needs to be captured by the entrance tank. The first approach approximates the scouring velocity in the inlet channel with the following equation: $$V_{min} = \sqrt{\frac{2}{3}\frac{\sqrt{Re_x}}{0.332}\frac{\rho_{p} - \rho_{H20}}{\rho_{H20}}\ g\ d_p\ tan\theta}$$

The second approach approximates the shear on the bottom of the inlet channel and uses a modified Shields diagram to determine the corresponding critical particle diameter. **Both approaches produced a critical particle diameter (2.373 mm and 7.5 mm respectively) that is larger than the typical size of sand. These results are not physically reasonable, and do not match observations made by plant operators in Gracias.** The key assumptions made in each approach should be re-examined or a new model should be developed. 

**Entrance Tank Kinetic Energy**

Wave action and splashing is another issue occuring in the entrance tank at the plant in Gracias, Honduras. The velocity and kinetic energy was calculated at various points within the entrance tank (Table 1): exiting the influent pipe, through the plate settlers, and at the shallow channel leading to the LFOM. 

*Table 1: The velocity and kinetic energy at various location in the entrance tank*

|Location|Velocity (m/s)|Kinetic Energy (cm)|
|:---:|:---:|:---:|
|Influent Pipe|1.91|18.6|
|Plate Settlers|0.16|0.1305|
|LFOM Channel|0.1593|0.1294|

The velocity exiting the influent pipe is much greater than other areas of the entrance tank. **This suggests that the kinetic energy exiting the influent pipe may be the primary source of wave action.** Various methods should be explored to reduce this kinetic energy. This design may include the following components:
- aiming the influent pipe at the end wall of the entrance tank
- use 2" pipes as diffusers
- placing obstacles along the side walls of entrance tank
- including a diffuser wall (the trash racks may assist with this)
- place an endcap on the influent pipe so that water does not flow straight down into the hopper, and cut a vertical slit down the side to act as a diffuser. 

Each component should be evaluated on its effectiveness at dissipating kinetic energy and its ease of construction.

**Next Steps:**
- review emails from APP for more insight on the entrance tank design
- start exploring different ways to reduce the kinetic energy in the entrance tank








# Python Calculations

In [5]:
!pip install aguaclara
from aguaclara.core.units import unit_registry as u
import aguaclara as ac
import numpy as np

# global variables
q = (60 * u.L/u.s).to(u.m**3/u.s)
T = 20 * u.degC
rho_H20 = ac.density_water(T)
rho_p = 2650 *u.kg/u.m**3 # density of sand
nu = ac.viscosity_kinematic_water(T) # kinematic viscosity
mu = nu*rho_H20 # dynamic viscosity of 
w_flocculator = 0.60 * u.m # the width of one flocculator at Gracias (inner width ie. does not include thickness of the concrete)
length_flocculator = 7.10 * u.m # the length of one flocculator channel at Gracias (inner length ie. does not include thickness of the concrete)
depth_flocculator = 2.17 * u.m  # at Gracias


## Design the Entrance Tank in Python

*The entrance tank will morph from option 2 (like Gracias) to option 3 (full length of the flocculator) to wider (to get even better removal of grit). Determine the number of hoppers depending on the L/W ratio.* 





In [30]:
# Design the entrance tank in python

# add safety factors?

# outer dimensions of the entrance tank
d_p = (0.25 * u.mm).to(u.m) # minimum grit diameter to be removed (arbitrary number chosen - depends on water source)
v_c = ((rho_p - rho_H20)*ac.GRAVITY*d_p**2)/(18*nu*rho_H20)
Re = v_c*d_p/nu
if Re > 2: # recalculate for transition flow (MWH textbook uses 2 as laminar boundary)
   v_c = (((rho_p - rho_H20)*ac.GRAVITY*d_p**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4) 
   Re = v_c*d_p/nu
area_ET = q/v_c
width_ET = w_flocculator
length_ET = ((area_ET/width_ET).magnitude) * u.m # Need to think about how the area taken up by the LFOM, trashracks, and influent pipe might fit into this
if length_ET > length_flocculator:
   length_ET = length_flocculator
   width_ET = area_ET/length_ET
#length_min = Figure out how much space the components inside the ET take up
#if length_ET < length_min:
   #length_ET = length_min

print('The length of the entrance tank is ',length_ET)
print('The width of the entrance tank is ', width_ET)

# determine the number and dimension of hoppers
LW_ratio_design = 1.2 # for long ETs this ratio could be increased to reduce the total number of hoppers required
width_hopper = width_ET
length_hopper = LW_ratio_design*width_hopper
n_hoppers = np.floor(length_ET/length_hopper) 
# redistribute extra space to each hopper equally
extraLength = (length_ET - n_hoppers*length_hopper)/n_hoppers #the extra length added to each hopper to fill the entire entrance tank
length_hopper = length_hopper + extraLength
LW_ratio_actual = length_hopper/width_hopper

# the minimum number of hoppers is 2 (either side of the trash racks) need to flesh out this scenario

print('There are ',n_hoppers.magnitude, 'hoppers with a length:width ratio of ', LW_ratio_actual)
print('The width of each hopper is ', width_hopper)
print('The length of each hopper is ',length_hopper)

# determine the hopper depth
#slope = 1/100 # z:x or z:y 
#depth_hopper = # the distance between the bottom of the ET and the top of the hopper
#diam_drainpipe = 







The length of the entrance tank is  3.147 meter
The width of the entrance tank is  0.6 meter
There are  4.0 hoppers with a length:width ratio of  1.311 dimensionless
The width of each hopper is  0.6 meter
The length of each hopper is  0.7866 meter


## Calculate the Required Length of the Entrance Tank

*Assume that the entrance tank has the same width as the flocculator and that the plant is running at full capacity. Calculations will focus on the inlet channel of the sedimentation tank, since high shear around baffles in the flocculator makes it less likely that particles will settle there.*

**Strategy**

1. Determine the diameter of the smallest particle that needs to be removed by the entrance tank.
    - **Approach 1:** Use a critical particle size of 0.1 mm recommended by Kawamura (pg. 416)
    - **Approach 2:** Calculate the diameter of the largest particle that will be scoured in the inlet channel of the sedimentation tank by rearranging the [equation](https://github.com/AguaClara/CEE4520/blob/master/Lectures/Extras/Sedimentation%20Extras.pptx) (slides 15-18) for scouring velocity:$$V_{min} = \sqrt{\frac{2}{3}\frac{\sqrt{Re_x}}{0.332}\frac{\rho_{p} - \rho_{H20}}{\rho_{H20}}\ g\ d_p\ tan\theta}$$
    - **Approach 3:** Calculate the shear on the bottom of the inlet channel of the sedimentation tank and use a modified Shields diagram to determine the critical particle size. 
1. Use the appropriate Stokes Law for laminar or turbulent flow to determine the corresponding critical velocity of the particle.

   $$**insert \ equations**$$
1. Calculate the required plan-view area of the entrance tank
   $$**insert \ equations**$$

### **Approach 1:** Use the 0.1 mm recommended particle diameter provided by Kawamura (pg. 416)

In [2]:
d_p = (0.1 * u.mm).to(u.m) # minimum grit diameter to be removed according to Kawamura
v_c = ((rho_p - rho_H20)*ac.GRAVITY*d_p**2)/(18*nu*rho_H20)
print('The cricitcal velocity is ',v_c.to(u.mm/u.s))

# check that flow is in laminar regime for Stokes' law to be valid
Re = v_c*d_p/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area = q/v_c
length_ET = area/w_flocculator
print('The required entrance tank length is ', length_ET)

The cricitcal velocity is  6.919 millimeter / second
The Reynolds number is  0.5322 dimensionless
The required entrance tank length is  14.45 meter


**Reflection:** The calculated critical velocity is 8.984 mm/s, which is very close to the 8 mm/s that was suggested by Monroe. The Reynolds number is well within the laminar regime so using Stokes' law was an appropriate decision. The entrance tank requires a length of 11.13 meters. For comparison, the length of one treatment train at Gracias (sed tanks + flocculators) is approxiamtely 15 m, and the width of one treatment train is about 7 m. Therefore, trying to capture such small particles takes up a considerable amount of space, and may drive up materials and construction costs. Additionally, an entrance tank of that size my block plant operator access to important parts of the plant, such as the first four baffles in the flocculator, where plant operators like to observe flocs. 

### **Approach 2:** Calculate the scouring velocity in the inlet channel of the sedimentation tanks






For open-channel flow, the [equation](https://github.com/AguaClara/CEE4520/blob/master/Lectures/Extras/Sedimentation%20Extras.pptx) (slides 15-18) describing scouring velocity is:

$$V_{min} = \sqrt{\frac{2}{3}\frac{\sqrt{Re_x}}{0.332}\frac{\rho_{p} - \rho_{H20}}{\rho_{H20}}\ g\ d_p\ tan\theta}$$

where

$Re_x$ = the Reynold's number based on the distance along the channel. Note that the lowest amount of shear occurs where the boundary layer switches from laminar to turbulent (ie. $Re_x$ = 500,000) (slide 15).

$\rho_{p}$ = density of the particle

$\rho_{H20}$ = density of the water

$g$ = acceleration due to gravity

$d_p$ = the particle diameter

$\theta$ = a parameter that is approximated from Shields Diagram (slide 18), assuming turbulent flow
     $$\theta =  tan^{-1}(0.056* \frac{3}{2}) = 4.8^\circ$$ 

Knowing the average velocity in the inlet channel, this equation can be rearranged to calculate the size of the largest particle that will be scoured. The inlet channel is designed with a sloped bottom such that it has a relatively constant velocity along the length of the channel. The simplest place to calculate the average velocity is the point where water enters the channel.  


In [3]:
# calculate the average velocity in the inlet channel to the sedimentation tanks, assuming that it is relatively constant due to sloped bottom geometry
theta = (4.8*u.deg).to(u.rad)
w_chan = 0.5 * u.m # width of the inlet channel to the sedimentation tanks
h_chan = 0.156 * u.m # the height of the inlet channel measured at the influent end
a_chan = w_chan*(h_chan - 0.1 * u.m) # The inlet channel to the sedimentation tanks have a 10 cm freeboard
v_chan = q/a_chan
print('The velocity in the influent channel of the sedimentation tank is ', v_chan)

# rearrange the scouring velocity equation to calculate the critical particle diameter
dp_2 = (3*0.332*rho_H20*v_chan**2)/(2*500000**0.5*(rho_p - rho_H20)*ac.GRAVITY*np.tan(theta))
print('The largest diameter particle that will be scoured by the inlet channel of the sedimentation tanks is ', dp_2.to(u.mm))

# calculate the terminal velocity of the particle using Stokes' Law assuming laminar flow
vc_2 = ((rho_p - rho_H20)*ac.GRAVITY*dp_2**2)/(18*nu*rho_H20)
print('The cricitcal velocity is ',vc_2.to(u.mm/u.s))

# check that flow is in laminar regime for Stokes' law to be valid
Re = vc_2*dp_2/nu
print('The Reynolds number is ', Re)
print('The calculation needs to repeated using Stokes Law for transition flow')

# calculate the terminal velocity of the particle assuming the transition/turbulent regime
v_c_transition = (((rho_p - rho_H20)*ac.GRAVITY*dp_2**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4)
print('The critical velocity assuming transitional flow is ', v_c_transition)

# check that flow is in the transition/turbulent regime
Re = v_c_transition*dp_2/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area2 = q/v_c_transition
length_ET2 = area2/w_flocculator # assume ET is same width as the flocculator
print('The entrance tank length is ', length_ET2)

The velocity in the influent channel of the sedimentation tank is  2.143 meter / second
The largest diameter particle that will be scoured by the inlet channel of the sedimentation tanks is  2.379 millimeter
The cricitcal velocity is  3915 millimeter / second
The Reynolds number is  7165 dimensionless
The calculation needs to repeated using Stokes Law for transition flow
The critical velocity assuming transitional flow is  0.3728 meter ** 1 / second
The Reynolds number is  682.2 dimensionless
The entrance tank length is  0.2682 meter ** 1


In [17]:
# This is scrap code: I'm backchecking this calculation with current Gracias dimensions
Length_ET = (16 * u.cm).to(u.m)
area = Length_ET*w_flocculator
v_c = q/area
d_p = (v_c**(1.4)*(13.9*mu**0.6*rho_H20**0.4)/((rho_p - rho_H20)*ac.GRAVITY))**(1/1.6) # uses transition equation
Re = v_c*d_p/nu

print('the critical particle size is ',d_p)
print('Re is ', Re)

# the holes on the trash rack appear to be about 12.8 (rounds to 13 mm in diameter), so any particles we are talking about here would pass through
# so theoretically, the current entrance tank at Gracias should be able to capture a 3.388 mm particle

the critical particle size is  0.003388 meter ** 1
Re is  2110 / meter ** 2.22045e-16


**Reflection:** The critical particle size is 2.373 mm and the required entrance tank length is 24 cm. We should be skeptical of these results, because the entrance tank at Gracias is already longer than 24 cm long, and plant operators are observing grit in both the flocculator and inlet to the sedimentation tanks. 

### **Aproach 3:** Calculate the bottom shear in the inlet channel and use a modified Shields Diagram

**Bottom Shear Stress**

We will assume a turbulent boundary layer on the bottom of the inlet channel to the sedimentation tanks, since achieving laminar flow in a water treatment plant takes considerable effort. Wall shear stress, $\tau_w$, in the turbulent regime can be modeled by the following [Equation:](http://www-mdp.eng.cam.ac.uk/web/library/enginfo/aerothermal_dvd_only/aero/fprops/introvisc/node8.html)

$$\tau_w = \frac{1}{2}\rho_{H20} U_ {\infty}^2 C_f $$

where 

$\rho_{H20}$ = density of water

$U_ {\infty}$ = free stream speed. 

$C_f$ = the skin friction

<br/>

In the turbulent regime,

$$C_f = \frac{0.0594}{Re_x^{0.2}}$$ 

where

$Re_x$ = the Reynolds number with respect to the distance along the channel

The shear is the lowest when the boundardy layer switches from laminar to turbulent. Thereofre $Re_x$ will be assumed to be 500,000 [(slide 15)](https://github.com/AguaClara/CEE4520/blob/master/Lectures/Extras/Sedimentation%20Extras.pptx).

<br/>

**Shields Diagram**

Shields diagram represents the empirical relationship between two dimensionless parameters that describe the incipient movement of particles ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 268): 

1) *The 'Shear' or 'Boundary' Reynolds Number*

$$Re_* = \frac{u_*D}{\nu}$$

where

$u_* = \sqrt{\frac{\tau_o}{\rho}}\ \ $"shear velocity"

$\rho$ = density of the fluid

D = particle diameter

$\nu$ = kinematic viscosity

<br/>

2) *Shields Parameter* ([Hydraulic](https://github.com/monroews/Hydraulics/raw/master/10_Channel_Design.pptx), slides 19-23)

$$\frac{\tau_o}{\Delta\rho gD} = \frac{2}{3}tan\theta$$

where

$\tau_o$ = shear stress

$\Delta \rho$ = the difference in density between the particle and water

$D$ = particle diameter

*Side note: the $\frac{2}{3}$ fraction on the right hand side may also depend on geometrical and dynamical effects ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 267) and multiplying the numerator and denominator by $D^2$ gives a ratio of fluid force on the particle to the weight of the particle (pg. 269).*

The original version of Shields Diagram is not particularly useful for determining the critical particle size that will be scoured in the inlet channel, because both shear stress and particle diameter occur as variables on both the x and y axis. 

However, Shields diagram can be 'recast' so that only one axis contains the variable $\tau_o$ and only the other axis contains the particle diameter D. These new axis are as follows ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 275): 

$$\tau_o(\rho /(g\Delta\rho)^2\mu^2)^{1/3}\ \ \  and\ \  \ D(\rho (g\Delta\rho)/\mu^2)^{1/3}$$

Figure 1 shows an updated version of Shields diagram reframed in terms of shear velocity and particle diameter at $20 ^\circ C$ ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), pg. 277). Note that the axis do not look like the ones described above, because constant values at the given temperature such as the acceleration due to gravity and dynamic viscosity have already been plugged in. 

![Shields Diagram](https://github.com/Emily-Wood/AguaClara/blob/master/Updated_Shields_Diagram%20.PNG?raw=true)


**Figure 1**: Shields diagram recast in terms of shear velocity and particle diameter at $20 ^\circ C$ ([Chapter 9](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf), page 277)

Using the calculated bottom shear stress, the modified Shields Diagram can be used to determine the critical particle size. 

**Sources:** 
1. Chapter 9 [Introduction to Fluid Motion, Sediment Transport and Current Generated Sedimentary Structures](https://ocw.mit.edu/courses/earth-atmospheric-and-planetary-sciences/12-090-introduction-to-fluid-motions-sediment-transport-and-current-generated-sedimentary-structures-fall-2006/course-textbook/ch9.pdf)
1. [Hydraulic Engineering: Channel Design Powerpoint](https://github.com/monroews/Hydraulics/raw/master/10_Channel_Design.pptx) (slides 19-23)









In [4]:
# calculate the average velocity in the inlet channel of the sedimentation tanks (ie. "free stream speed")
L_chan = 12 * u.m
w_chan = 0.5 * u.m
h_chan = 0.156 * u.m # the height of the channel at the influent end
a_chan = w_chan*(h_chan - 0.1 * u.m) # The inlet channel to the sedimentation tanks have a 10 cm freeboard
v_chan = q/a_chan

#Re_x = 500000
Re_x = v_chan*L_chan/nu # choose the far end of the channel for a conservative estimate of tau
print('The Reynolds number with respect to the channel length is ',Re_x)
C_f = 0.0594/(Re_x**0.2)
tau = 0.5*rho_H20*v_chan**2*C_f

u_star = (tau/rho_H20)**0.5
print('The shear velocity is ', u_star.to(u.cm/u.s))

# According to the recast Shields Diagram, the corresponding critical particle diameter is approximately 1.3 cm
dp_3 = (7.5 * u.mm).to(u.m)
print('Using the modified Shields diagram, this corresponds to a particle diameter of ', dp_3.to(u.mm))

# Calculate the terminal velocity of the particle assuming transitional/turbulent flow
v_c_transition2 = (((rho_p - rho_H20)*ac.GRAVITY*dp_3**1.6)/(13.9*mu**0.6*rho_H20**0.4))**(1/1.4) 
print('The cricitcal velocity is ', v_c_transition2)

# check that flow is in the transitional/turbulent regime
Re = v_c_transition2*dp_3/nu
print('The Reynolds number is ', Re)

# calculate the required ET length
area3 = q/v_c_transition2
length_ET3 = area3/w_flocculator
print('The entrance tank length is ', length_ET3)

The Reynolds number with respect to the channel length is  1.978e+07 dimensionless
The shear velocity is  6.882 centimeter / second
Using the modified Shields diagram, this corresponds to a particle diameter of  7.5 millimeter
The cricitcal velocity is  1.385 meter ** 1 / second
The Reynolds number is  7990 dimensionless
The entrance tank length is  0.07221 meter ** 1


**Reflection:** With a Reynolds number of 500,000 the shear velocity is 9.942 cm/s. The corresponding critical particle diameter of 1.3 cm is clearly not a physically reasonable result. A piece of grit this large probably would not even make it into the plant, and would certainly be caught by the trash racks. Perhaps the equations selected to model the bottom shear in the inlet channel are not applicable to this situation, or the wrong Reynolds number was selected. It is also really hard to read the recast Shields diagram without gridlines - a small amount of reading error could lead to an order of magnitude difference in particle diameter. The initial assumption of this calculation should be checked, and a different method for approximating the shear stress on the bottom of the inlet channel could be considered. 

To troubleshoot this approach, I first tried calculating the Reynold's number for the channel directly instead of assuming that it is 500,000. I used the total channel length as the important length, because it would lead to the most conservative measure of shear stress (and less shear means that it is more difficult to scour particles). This led to a new Reynolds number of 25,620,000 (This number is really high, I'm not sure if it is reasonable for the open channel flow in this application) and a shear velocity of 6.707 cm/s. These numbers produce an approximate critical particle size of 7.5 mm, and an entrance tank length of 6.5 cm. This is still a very large particle size. 

Approach 3 is also less reliable than approach 2, because it relies on reading an empirical graph precisely rather than calculating particle size directly using first principles. 




## Analyze the Kinetic Energy in the Entrance Tank at the Gracias, Honduras Plant

### Calculate the kinetic energy and velocity exiting the influent pipe at Gracias


In [ ]:
ID = 0.2 * u.m # inner diameter of influent pipe at Gracias
a = 0.25*np.pi*ID**2
v_influent = q/a
KE_influent = v_influent**2/(2*ac.GRAVITY)

print('The velocity exiting the influent pipe is ', v_influent)
print('The kinetic energy exiting the influent pipe is ', KE_influent.to(u.cm))

The velocity exiting the influent pipe is  1.91 meter / second
The kinetic energy exiting the influent pipe is  18.6 centimeter


### Calculate Ke = v^2/2g through the plate settlers at Gracias


In [ ]:
# calculate the kinetic energy through the plate settlers at Gracias

s = 0.0250 * u.m # edge-to-edge distance between plate settlers (ie. does not include plate thickness)
n = 24 # number of plate settlers in entrance tank
w = 0.6 * u.m # plate settler width
area = s*(n+1)*w
v = q/area
Ke = v**2/(2*ac.GRAVITY)

print('The hypothetical velocity through the plate settlers assuming uniform flow is ', v)
print('The kinetic energy through the plate settlers at Gracias is ', Ke.to(u.cm))

The hypothetical velocity through the plate settlers assuming uniform flow is  0.16 meter / second
The kinetic energy through the plate settlers at Gracias is  0.1305 centimeter


### Calculate the velocity through the channel leading to the LFOM

In [ ]:
# calculate the velocity through the channel leading to the LFOM

d = 0.6278 * u.m # depth of water immediately after the plate settlers measured to the top hole on the LFOM (ie. assuming holes are completely covered at 60 L/s flow)
w = 0.6 * u.m # width of ET
area_LFOMchan = d*w
v_LFOMchan = q/area_LFOMchan
Ke_LFOMchan = v_LFOMchan**2/(2*ac.GRAVITY)

print('The velocity in the channel leading to the LFOM is ', v_LFOMchan)
print('The kinetic energy in the channel leading to the LFOM ', Ke_LFOMchan.to(u.cm))

The velocity in the channel leading to the LFOM is  0.1593 meter / second
The kinetic energy in the channel leading to the LFOM  0.1294 centimeter


**Reflection:** It makes sense that neither the velocity or Ke change very much in these two calculations, because the cross-sectional area for flow didn't really change, and v = q/a. The Ke through the plate settlers is well below 5 cm (5 cm would be considered very high)

# Information and Notes

## Executive Summaries

### Executive Summary: June 22 - June 28 2020  
The trash rack and plate settlers in the entrance tank at Gracias have been unsuccessful at capturing grit and preventing it from traveling to the rest of the plant. This grit causes maintenance issues by settling in the flocculator and the channel leading to the sedimentation tanks. To solve this problem, the plate settlers could be replaced with a more traditional horizontal settling basin. A series of 4-sided hoppers fitted with a drainpipe would make it easy to remove grit from the bottom of the chamber via gravity. However, Kawamura advises against the use of square tanks, due to their poor flow patterns:

> “The square tank is satisfactory for the purpose of wastewater treatment but does not perform well as a water treatment grit chamber. The main problem with the detritus [square] tank is a function of its physical characteristics. The short tank length and the uneven inflow to the tank cause severe flow short-circuiting. … To provide good flow characteristics for grit settling, the chamber should be rectangular with a contracted inlet (pg. 414).” 

**To settle particles greater than 0.1 mm in diameter, the entrance tank at Gracias would need to be 11.13 m in length.** This calculation was done under the assumption of a 60 L/s flow rate, and that the entrance tank is the same width as the flocculator.  According to Kawamura, sand and silt smaller than 0.1 mm do not pose a significant risk to downstream processes (pg. 416). For context, a single treatment train (excluding the filters) at Gracias has an approximate 15 m x 7 m area. Increasing the area of the entrance tank would drive up construction costs and may block plant operator access to important parts of the plant. For example, observing the flocs in the first 4 baffles of the flocculator is one of the most reliable indications of plant performance. 

The required plan-view area of the entrance tank could be reduced by increasing the critical particle size that needs to be captured. Kawamura recommended a 0.1 mm diameter based on pump and piping damage in downstream processes. However, the predominant issue in Gracias, Honduras is grit settling in the flocculator and inlet channel to the sedimentation tanks.

The velocities going around the bottom bend of the flocculator are comparatively higher than velocities in the entrance tank. This suggests that particles larger than 0.1 mm could be scoured from the bottom of the flocculator, and that the entrance tank may be designed for a larger critical particle diameter. **The shear on the bottom of the flocculator should be calculated to determine the largest particle that can be scoured.**

Wave action and splashing is another issue occuring in the entrance tank at the plant in Gracias, Honduras. The velocity and kinetic energy was calculated at various points within the entrance tank (Table 1): exiting the influent pipe, through the plate settlers, and at the shallow channel leading to the LFOM. 

*Table 1: The velocity and kinetic energy at various location in the entrance tank*

|Location|Velocity (m/s)|Kinetic Energy (cm)|
|:---:|:---:|:---:|
|Influent Pipe|1.91|18.6|
|Plate Settlers|0.16|0.1305|
|LFOM Channel|0.1593|0.1294|

The velocity exiting the influent pipe is much greater than other areas of the entrance tank. **This suggests that the kinetic energy exiting the influent pipe may be the primary source of wave action.** Various methods should be explored to reduce this kinetic energy. This design may include the following components:
- aiming the influent pipe at the end wall of the entrance tank
- use 2" pipes as diffusers
- placing obstacles along the side walls of entrance tank
- including a diffuser wall (the trash racks may assist with this)
- place an endcap on the influent pipe so that water does not flow straight down into the hopper, and cut a vertical slit down the side to act as a diffuser. 

**Each component should be evaluated on its effectiveness at dissipating kinetic energy and its ease of construction.** 


## Information sources
1.  [Capstone Design: Improved Entrance Tank Design](https://github.com/AguaClara/CEE4520/blob/master/Previous_Final_Projects/2019F/Improved_Entrance_Tank.ipynb)
1.   MWH textbook
1.   [Susumu Kawamura textbook](https://photos.app.goo.gl/6KWgu6m9uSKmRAKH9)
1.   [Sed tank design code example](https://colab.research.google.com/github/AguaClara/CEE4520/blob/master/Unit_Process_Designs/Sed.ipynb)
1. [Gracias CAD Design](https://cad.onshape.com/documents/cb0559d6c3bd85907f8704d3/w/8b31f12d476b14cf91bd29d9/e/6d7584b2804b3c166ba1e991)
1. [Hydraulic Engineering: Channel Design Powerpoint](https://github.com/monroews/Hydraulics/raw/master/10_Channel_Design.pptx) (slides 19-23)
1. Chapter 12: Stable Channel Design Functions in [HEC-RAS River Analysis System Hydraulic Reference Manual](https://www.hec.usace.army.mil/software/hec-ras/documentation/HEC-RAS%205.0%20Reference%20Manual.pdf)  (page 12-21)

## Problems with the current design

1.   Explore options to reduce the kinetic energy of the incoming water. This results in lots of wave action and splashing. Perhaps inject the water deeper into the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface. Or direct the water against the wall of the tank? Perhaps  2" diameter diffusers (so that clogging isn't a problem) in the vertical entrance pipe with the diffusers pointing upstream toward the end wall of the entrance tank? 
1.   Plate settlers don't work because the velocities are too high and flow isn't distributed at all unformly through the plates. Recommend either improving flow distribution (see 4520 capstone design project) or reverting to a simple horizontal flow tank to capture grit. The simple horizontal sed tank should be explored as the simplest option.
1.   Velocity through the bar screen is too high. Increase the area for the bar screen and include a minimum of 2 bar screens and 3 tracks for inserting the screens so that one can be removed and cleaned and replaced. Screen can be angled to enable it to have more surface area.
1.   Velocity of approach for the LFOM is too high causing wave action. See if we can predict the wave action that is observed at Gracias. The waves are simple a conversion of kinetic energy into potential energy, $v=\sqrt{2gh}$. 

The new entrance tank will likely take more horizontal space OR it is likely that it simply is the full depth of the flocculator with several hoppers that drain any accumulated sediment.

## Strategy

1.  Design the ET in python
1.  Draw the ET in Onshape
1.  Transition on the python code to Feature Script
1.  Intergrate the new ET with the flocculator, LFOM, and CDC



## Notes from meeting with Monroe








### Wed. June 24
- clarification: we are concerned with splashing everywhere in the ET
- start with worst case scenario, if the pipe coming into the entrance tank was directed right towards the surface. Take the velocity coming out of influent pipe, and find the velocity flowing uniformly between the plates. There is very little headloss between the plates, so it's difficult to get an even distribution of flow
- From powerpoint about turbulence:
  - b is the width of the jet
  - Vc is the velocity at the center of the jet
  - s equals the distance forward
  - there is a pproximately a 1/10 ratio between s and b, ie. b = 0.116s or something like that (number might not be exactly correct). Suppose the orifices on a trash rack are spaced out a centimeter. This means that 10 cm out, all of the jets will have merged.
- We want to create many small eddies, because they dissipate the fastest, the trash rack can possibly help with this.
  - Make the trash rack taller and potentially at a slope to increase surface area.
  - at the far end of the flocculator, the channel is 2 m deep (we think), upstream add 50 cm headloss, then add 10 cm drop in the LFOM, and then an additional 20 cm for a total of 2.8 m height in the entrance tank. The trash racks would be placed after the first hopper and would go as deep as the top of the hopper. There is a physical limitation to how deep the trash rack can be, because the operators have to be able to lift it to clean it.
- There are going to be 4 sided hoppers, need to determine how many and the length of the entrance tank. The number of drains depends on the size of particles. Confirm capture velocity from Kawamura textbook being 8 mm/s and this will set the length of the sed tank
- the influent pipe would have a firnco coupling so it can be taken out, and possibly capped at the bottom end to prevent water from flowing straight down into the first hopper. This would be bad because it would not promote even flow throughout the tank. Influent pipe could also have a slot on the side instead of diffusers.
- the eddy velocity in a pipe is roughly 10% of the mean velocity

**Action Items:**
- Monroe will send the pictures of the Kawamura textbook **complete**
- Calculate v^2/2g for the plate settlers as Gracias. 5 cm would be a lot. **complete**
- Calculate the velocity in the channel leading to the LFOM (where it gets a lot more shallow really fast) **complete**
- Calculate how long the ET would need to be for a 60 L/s plant when designed as a horizontal sedimentation chamber. Assume the same flocculator width as Gracias. **complete**

**Reflections & Future Questions**
- Will flow expansion around a barrier dissipate more kinetic energy than water flowing through a barrier with a bunch of hole drilled into it? 
- Do non-AguaClara entrance tanks have similar problems? Can we learn anything from how problems have been dealt with in the past? 
- is the energy dissipation rate of water coming out of the influent pipe related to the energy dissipation rate of the turbulent eddies?
- For predicting wave action around LFOM, what observations at Gracias should this be based off of?
- are there any drawbacks to injecting the water deeper in the entrance tank?
- the bottom geometry of the entrance tank should be selected to ensure that maintenance and the removal of grit is simple and effective. 

**Kawamura Textbook Notes:**

- "a hydraulic scale model conducted by the author clearly demonstrated that when the screen is installed properly, it functions as an inlet diffuser wall"
- "The square tank is satisfactory for the purpose of wastewater treatment but does not perform well as a water treatment grit chamber. The main problem with the detritus [square] tank is a function of its physical characteristics. The short tank length and the uneven inflow to the tank cause severe flow short-circuiting. ... To provide good flow characteristics for grit settling, the chamber should be rectangular with a contracted inlet."

**MWH Textbook Notes:**
"Although coarse screens are used with river intakes, a fine debris screen
with approximately 20-mm openings is often provided at the front end
of the presedimentation tank. Because the screen also acts as an effective
diffuser wall, it should be installed close to the tank inlet. If a separate
diffuser wall is specified, the total area of openings at the wall should be
about 15 percent of the tank cross-sectional area" (pg. 667)

## Task: Reduce the kinetic energy of the influent water

**Problem:** The high kinetic energy of the influent water causes lots of wave action and splashing 

**Possible Solutions**
1. inject the water deeper into the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface.
1. direct the water against the wall of the tank
1. use 2" diameter diffusers (so that clogging isn't a problem) in the vertical entrance pipe with the diffusers pointing upstream toward the end wall of the entrance tank



### Task 1 Solution 1: Inject water deeper in the entrance tank so that turbulent eddies are somewhat dissipated by the time they get to the water surface

**Edit 6/24: not sure that we really need to any math to determine how deep the influent tube should be. would probably just position it as low as possible. Ask Monroe later.**

- Define "somewhat dissipated."
  - From AC textbook: "Eddy turnover time, $t_{eddy}$, is the time it takes for the eddy to travel a distance equal to its length-scale. Thus the eddy turnover time provides a good estimate of the time required for mixing to occur at the length scale of the eddy. **We assume that the energy of the large eddy is dissipated into smaller length scales in the time $t_{eddy}$:**"

- Which equations govern the dissipation of turbulent eddies?
$$t_{eddy} \approx \frac{L_{eddy}}{v_{eddy}}$$

   The rate of energy loss to smaller scales
$$\bar\varepsilon \approx\frac{v_{eddy}^3}{L_{eddy}}$$

   and therefore...

$$v_{eddy} \approx \left( \bar\varepsilon \, L_{eddy} \right)^\frac{1}{3}$$

- "The largest eddies are limited in size by the smallest dimension normal to the direction of flow"

   *Therefore the length scale of a turbulent eddy in the entrance tank is either the vertical distance between the water surface and the 'baffle wall', or the width of the entrance tank (whichever distance is smaller).*???

- How is water currently being injected?
  - see onshape drawing, there's a tee at the end of the pipe so water flows out horizontally towards the side walls of the entrance tank

- For reference: The form of energy loss from the influent pipe is expansion, (ie. the following equations are relevant)

$$h_e = K\frac{\bar v_{out}^2}{2g}$$

$$\bar\varepsilon = K\frac{\bar v_{out}^3}{2H}$$

$$G_{CS} = \bar v_{out}\sqrt{\frac{K\bar v_{out}}{2H\nu}}$$

 
- Determine the injection depth needed so that turbulent eddies are "somewhat dissipated"
  
